In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv",
    index_col=0
    )
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336776 entries, 1 to 336776
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       336776 non-null  int64  
 1   month      336776 non-null  int64  
 2   day        336776 non-null  int64  
 3   dep_time   328521 non-null  float64
 4   dep_delay  328521 non-null  float64
 5   arr_time   328063 non-null  float64
 6   arr_delay  327346 non-null  float64
 7   carrier    336776 non-null  object 
 8   tailnum    334264 non-null  object 
 9   flight     336776 non-null  int64  
 10  origin     336776 non-null  object 
 11  dest       336776 non-null  object 
 12  air_time   327346 non-null  float64
 13  distance   336776 non-null  int64  
 14  hour       328521 non-null  float64
 15  minute     328521 non-null  float64
dtypes: float64(7), int64(5), object(4)
memory usage: 43.7+ MB


### Seleção de colunas & observações

1. Colunas de interesse:
- year
- month
- day
- hour
- minute
- dep_delay
- arr_delay
- carrier
- flight
- air_time
- distance
- origin
- dest
2. Campos que não devem ser nulos:
- carrier
- flight
- year
- hour
- minute
3. Campos que devem ser positivos:
- air_time

In [4]:
usecols=["year", "month",  "day", "hour", "minute","arr_delay","carrier","flight","air_time","distance", "origin", "dest"]

In [5]:
df_raw = df.loc[
    (~df["carrier"].isna()) \
    & (~df["flight"].isna()) \
    & (~df["year"].isna()) \
    & (~df["hour"].isna()) \
    & (~df["minute"].isna()) \
    & (~df["month"].isna()) \
    & (~df["day"].isna()) \
    & (df["air_time"] > 0)
].loc[:, usecols]

In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327346 entries, 1 to 336770
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       327346 non-null  int64  
 1   month      327346 non-null  int64  
 2   day        327346 non-null  int64  
 3   hour       327346 non-null  float64
 4   minute     327346 non-null  float64
 5   arr_delay  327346 non-null  float64
 6   carrier    327346 non-null  object 
 7   flight     327346 non-null  int64  
 8   air_time   327346 non-null  float64
 9   distance   327346 non-null  int64  
 10  origin     327346 non-null  object 
 11  dest       327346 non-null  object 
dtypes: float64(4), int64(5), object(3)
memory usage: 32.5+ MB


In [7]:
df_raw.drop_duplicates(inplace=True)

In [8]:
df_raw = df_raw.astype("object")

In [9]:
tmp = df.loc[:, usecols].copy()
tmp = tmp[tmp["air_time"]>0]
for col in ["carrier","flight", "year", "month", "day" ,"hour", "minute"]:
    tmp_df = tmp.loc[~df[col].isna()]
    tmp = tmp_df.copy()

In [10]:
tmp.drop_duplicates(inplace=True)

In [12]:
print(tmp.shape[0] == df_raw.shape[0])
print(df_raw.shape)

True
(327346, 12)


### Criar uma coluna de date_time

In [12]:
df_raw["date_time"] =  pd.to_datetime(df_raw[["year", "month", "day", "hour", "minute"]],  dayfirst=True)

In [13]:
df_raw["date_time"]

1        2013-01-01 05:17:00
2        2013-01-01 05:33:00
3        2013-01-01 05:42:00
4        2013-01-01 05:44:00
5        2013-01-01 05:54:00
                 ...        
336766   2013-09-30 22:40:00
336767   2013-09-30 22:40:00
336768   2013-09-30 22:41:00
336769   2013-09-30 23:07:00
336770   2013-09-30 23:49:00
Name: date_time, Length: 327346, dtype: datetime64[ns]

In [14]:
usecols2 =["date_time", "arr_delay","carrier","flight","air_time","distance", "origin", "dest" ]

### Renamear colunas

In [15]:
new_columns = ["data_hora", "atraso_chegada", "companhia", "id_voo","tempo_voo", "distancia", "origem", "destino"]

In [16]:
columns_map = {usecols2[i]: new_columns[i] for i in range(len(usecols2))}
columns_map

{'date_time': 'data_hora',
 'arr_delay': 'atraso_chegada',
 'carrier': 'companhia',
 'flight': 'id_voo',
 'air_time': 'tempo_voo',
 'distance': 'distancia',
 'origin': 'origem',
 'dest': 'destino'}

In [17]:
df_work = df_raw.loc[:, usecols2].copy()
df_work.rename(columns=columns_map, inplace=True)
df_work.head()

,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,destino
1,2013-01-01 05:17:00,11.0,UA,1545,227.0,1400,EWR,IAH
2,2013-01-01 05:33:00,20.0,UA,1714,227.0,1416,LGA,IAH
3,2013-01-01 05:42:00,33.0,AA,1141,160.0,1089,JFK,MIA
4,2013-01-01 05:44:00,-18.0,B6,725,183.0,1576,JFK,BQN
5,2013-01-01 05:54:00,-25.0,DL,461,116.0,762,LGA,ATL


### Definir tipos das variáveis

In [18]:
df_work.dtypes

data_hora         datetime64[ns]
atraso_chegada            object
companhia                 object
id_voo                    object
tempo_voo                 object
distancia                 object
origem                    object
destino                   object
dtype: object

In [19]:
df_work["distancia"] = df_work.loc[:,"distancia"].astype(float)
df_work["companhia"] = df_work.loc[:,"companhia"].astype(str)
df_work["id_voo"] = df_work.loc[:,"id_voo"].astype(str)
df_work["atraso_chegada"] = df_work.loc[:,"atraso_chegada"].astype(str)
df_work["origem"] = df_work.loc[:,"origem"].astype(str)
df_work["destino"] = df_work.loc[:,"destino"].astype(str)

In [20]:
df_work.dtypes

data_hora         datetime64[ns]
atraso_chegada            object
companhia                 object
id_voo                    object
tempo_voo                 object
distancia                float64
origem                    object
destino                   object
dtype: object

### Tratamento strings

Escrever e aplicar uma função para:
- remover todos os caracteres eespeciais
- padronizar uppercase

In [21]:
import re

def padroniza_str(obs):
    return re.sub('[^A-Za-z0-9]+', '', obs.upper())

In [22]:
padroniza_str("ahsuw! @ ++  ~ ç 3n!!rrr")

'AHSUW3NRRR'

In [23]:
df_work["companhia"] = df_work.loc[:,"companhia"].apply(lambda x: padroniza_str(x))
df_work["id_voo"] = df_work.loc[:,"id_voo"].apply(lambda x: padroniza_str(x))
df_work["origem"] = df_work.loc[:,"origem"].apply(lambda x: padroniza_str(x))
df_work["destino"] = df_work.loc[:,"destino"].apply(lambda x: padroniza_str(x))

In [24]:
df_work.head()

,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,destino
1,2013-01-01 05:17:00,11.0,UA,1545,227.0,1400.0,EWR,IAH
2,2013-01-01 05:33:00,20.0,UA,1714,227.0,1416.0,LGA,IAH
3,2013-01-01 05:42:00,33.0,AA,1141,160.0,1089.0,JFK,MIA
4,2013-01-01 05:44:00,-18.0,B6,725,183.0,1576.0,JFK,BQN
5,2013-01-01 05:54:00,-25.0,DL,461,116.0,762.0,LGA,ATL


In [25]:
df_work.to_csv("base_tratada.csv", index=False)

### Validações

1. Validar se as colunas de interesse estão na base.
2. Validar se a seleção de observações não nulas e não positivas não exclui mais do que 5% da base.
3. Utilizar a função logging para armarzenar os erros e a quantidade de registros inseridos.

In [26]:
import logging

logging.basicConfig(filename='flights_pipe.log', level=logging.INFO) #vai criar um arquivo para os logs
logger = logging.getLogger()

In [27]:
logger.info(f'Inicio da execução ; {datetime.datetime.now()}')

In [28]:
#Gera uma falha no processo
if not set(usecols).issubset(set(df.columns)):
    logger.error(f"Mudança de schema; {datetime.datetime.now()} ")
    raise Exception("Mudança de schema")

In [29]:
1 - len(df_raw)/len(df)

0.02800080765850299

In [30]:
if 1 - len(df_raw)/len(df) > 0.05:
    logger.warning(f"Muitas observações perdidas na seleção; {datetime.datetime.now()} ")